In [1]:
! pip install -q git+https://github.com/huggingface/transformers.git
! pip install -q git+https://github.com/huggingface/datasets.git "dill<0.3.5" seqeval

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264

# PREPROCESSING DATA

In [2]:
from datasets import load_metric
from transformers import TrainingArguments, Trainer
from transformers import LayoutLMv3ForTokenClassification,AutoProcessor
from transformers.data.data_collator import default_data_collator
import torch
import pandas as pd
import numpy as np
import os
import argparse
from datasets.features import ClassLabel
from transformers import AutoProcessor
from sklearn.model_selection import train_test_split
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D, Dataset
from datasets import Image as Img
from PIL import Image

In [ ]:
def read_text_file(file_path):
  with open(file_path,'r') as f:
    return (f.readlines())

def prepare_examples(examples):
  images = examples[image_column_name]
  words = examples[text_column_name]
  boxes = examples[boxes_column_name]
  word_labels = examples[label_column_name]
  encoding = processor(images,words,boxes=boxes,word_labels=word_labels,truncation=True,padding='max_length')
  return encoding

def filter_out_unannotated(example):
  tags = example['ner_tags']
  return not all([tag==label2id['O'] for tag in tags])

In [ ]:
input_path = "/content/drive/MyDrive/intellectAI/layoutLM_input_format/"

files = dict()
files['train_box']  = read_text_file(input_path+'all_box.txt')
files['train_image'] = read_text_file(input_path+'all_image.txt')
files['train'] = read_text_file(input_path+'all.txt')

print(len(files['train_box']),len(files['train_image']),len(files['train']))

4818 4818 4818


In [ ]:
images = dict()
for i, row in enumerate(files['train_image']):
  if row != '\n':
    image_name = row.split('\t')[-1]
    images.setdefault(image_name.replace('\n', ''), []).append(i)

In [ ]:
words, bboxes, ner_tags, image_path = [], [], [], []
for image, rows in images.items():
  words.append([row.split('\t')[0].replace('\n', '') for row in files['train'][rows[0]:rows[-1]+1]])
  ner_tags.append([row.split('\t')[1].replace('\n', '') for row in files['train'][rows[0]:rows[-1]+1]])
  bboxes.append([box.split('\t')[1].replace('\n', '') for box in files['train_box'][rows[0]:rows[-1]+1]])
  image_path.append(f"{input_path}{image}")

In [ ]:
labels = list(set([tag for doc_tag in ner_tags for tag in doc_tag]))
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}

In [ ]:
dataset_dict = {
        'id': range(len(words)),
        'tokens': words,
        'bboxes': [[list(map(int, bbox.split())) for bbox in doc] for doc in bboxes],
        'ner_tags': [[label2id[tag] for tag in ner_tag] for ner_tag in ner_tags],
        'image': [Image.open(path).convert("RGB") for path in image_path]
    }

In [ ]:
len(dataset_dict['id'])

37

In [ ]:
features = Features({
        'id': Value(dtype='string', id=None),
        'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
        'bboxes': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
        'ner_tags': Sequence(feature=ClassLabel(num_classes=len(labels), names=labels, names_file=None, id=None), length=-1, id=None),
        'image': Img(decode=True, id=None)
    })

In [ ]:
full_data_set = Dataset.from_dict(dataset_dict, features=features)
dataset = full_data_set.train_test_split(test_size=0.1)
dataset["train"] = dataset["train"].filter(filter_out_unannotated)
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

Filter:   0%|          | 0/33 [00:00<?, ? examples/s]

In [ ]:
features = dataset["train"].features
column_names = dataset["train"].column_names
image_column_name = "image"
text_column_name = "tokens"
boxes_column_name = "bboxes"
label_column_name = "ner_tags"

In [ ]:
features = Features({
        'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
        'input_ids': Sequence(feature=Value(dtype='int64')),
        'attention_mask': Sequence(Value(dtype='int64')),
        'bbox': Array2D(dtype="int64", shape=(512, 4)),
        'labels': Sequence(ClassLabel(names=labels)),
    })

In [ ]:
train_dataset = dataset["train"].map(
        prepare_examples,
        batched=True,
        remove_columns=column_names,
        features=features,
    )
eval_dataset = dataset["test"].map(
        prepare_examples,
        batched=True,
        remove_columns=column_names,
        features=features,
    )
train_dataset.set_format("torch")
eval_dataset.set_format("torch")

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [ ]:
OUTPUT_PATH = '/content/drive/MyDrive/intellectAI/processed_data/'
train_dataset.save_to_disk(f'{OUTPUT_PATH}train_split')
eval_dataset.save_to_disk(f'{OUTPUT_PATH}eval_split')
dataset.save_to_disk(f'{OUTPUT_PATH}raw_data')

Saving the dataset (0/1 shards):   0%|          | 0/33 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/33 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4 [00:00<?, ? examples/s]

# TRAINING

In [3]:
from datasets import load_metric
from transformers import TrainingArguments, Trainer
from transformers import LayoutLMv3ForTokenClassification,AutoProcessor
from transformers.data.data_collator import default_data_collator
import torch
from datasets import load_from_disk

In [ ]:
input_path = '/content/drive/MyDrive/intellectAI/processed_data/'
train_dataset = load_from_disk(f'{input_path}train_split')
eval_dataset = load_from_disk(f'{input_path}eval_split')

In [ ]:
train_dataset

Dataset({
    features: ['pixel_values', 'input_ids', 'attention_mask', 'bbox', 'labels'],
    num_rows: 33
})

In [ ]:
label_list = train_dataset.features["labels"].feature.names
num_labels = len(label_list)
label2id, id2label = dict(), dict()
for i, label in enumerate(label_list):
    label2id[label] = i
    id2label[i] = label

In [ ]:
metric = load_metric("seqeval")
return_entity_level_metrics = False
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    # Remove ignored index (special tokens)
    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    results = metric.compute(predictions=true_predictions, references=true_labels,zero_division='0')
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

<ipython-input-10-a14127dda92b>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base",id2label=id2label,label2id=label2id)
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
NUM_TRAIN_EPOCHS = 150
PER_DEVICE_TRAIN_BATCH_SIZE = 1
PER_DEVICE_EVAL_BATCH_SIZE = 1
LEARNING_RATE = 4e-5
training_args = TrainingArguments(output_dir='test',
                                  # max_steps=1500,
                                  num_train_epochs=NUM_TRAIN_EPOCHS,
                                  logging_strategy="epoch",
                                  save_total_limit=1,
                                  per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
                                  per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH_SIZE,
                                  learning_rate=LEARNING_RATE,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  # eval_steps=100,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1")

In [ ]:
# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:830: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.003200,1.398867,0.418605,0.600000,0.493151,0.833333
2,0.110500,0.873516,0.425000,0.566667,0.485714,0.856838
3,0.042800,0.985860,0.394737,0.500000,0.441176,0.852564
4,0.012200,0.944259,0.418605,0.600000,0.493151,0.858974
5,0.010000,1.074378,0.500000,0.600000,0.545455,0.865385
6,0.006900,1.225387,0.435897,0.566667,0.492754,0.839744
7,0.011600,1.115585,0.545455,0.600000,0.571429,0.858974
8,0.020900,0.874863,0.425000,0.566667,0.485714,0.856838
9,0.012800,1.065724,0.414634,0.566667,0.478873,0.841880
10,0.004600,1.031356,0.485714,0.566667,0.523077,0.861111


/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:830: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:830: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:830: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:830: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:830: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:

TrainOutput(global_step=4950, training_loss=0.003598161418099107, metrics={'train_runtime': 2245.6797, 'train_samples_per_second': 2.204, 'train_steps_per_second': 2.204, 'total_flos': 1314119570227200.0, 'train_loss': 0.003598161418099107, 'epoch': 150.0})

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:830: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


{'eval_loss': 0.8960486054420471,
 'eval_precision': 0.7407407407407407,
 'eval_recall': 0.6666666666666666,
 'eval_f1': 0.7017543859649122,
 'eval_accuracy': 0.9081196581196581,
 'eval_runtime': 0.4535,
 'eval_samples_per_second': 8.821,
 'eval_steps_per_second': 8.821,
 'epoch': 150.0}

In [ ]:
OUTPUT_DIR = "/content/drive/MyDrive/intellectAI/model/"
torch.save(model,f'{OUTPUT_DIR}layoutlmv3_final.pth')

# INFERENCE

In [11]:
! sudo apt install tesseract-ocr
! sudo apt install libtesseract-dev
! pip install pytesseract
!python -m pip install -q 'git+https://github.com/facebookresearch/detectron2.git'
#! pip install -q git+https://github.com/huggingface/transformers.git
#! pip install -q git+https://github.com/huggingface/datasets.git "dill<0.3.5" seqeval

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.

In [12]:
import os
import torch
import warnings
from PIL import Image

In [50]:
import pandas as pd
import numpy as np
import torch
import os
import sys
import json
import logging

from PIL import Image, ImageDraw, ImageFont
from numpy.random import randint
from transformers import AutoProcessor

import warnings
import gc
import csv
warnings.filterwarnings('ignore')

In [14]:
def random_color():
  return np.random.randint(0,255,3)
  
def normalize_box(bbox,width,height):
  return [
          int(bbox[0]*(1000/width)),
          int(bbox[1]*(1000/height)),
          int(bbox[2]*(1000/width)),
          int(bbox[3]*(1000/height)),
  ]

def compare_boxes(b1,b2):
  b1 = np.array([c for c in b1])
  b2 = np.array([c for c in b2])
  equal = np.array_equal(b1,b2)
  return equal

def mergable(w1,w2):
  if w1['label'] == w2['label']:
    threshold = 7
    if abs(w1['box'][1] - w2['box'][1]) < threshold or abs(w1['box'][-1] - w2['box'][-1]) < threshold:
      return True
    return False
  return False

In [51]:
def inference(img_name,test_dir,output_path,model):
  imag_path = test_dir+img_name
  os.system(f'tesseract "{imag_path}" {img_name}_output -l eng tsv')
  inference_processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)
  ocr_df = pd.read_csv(f"{img_name}_output.tsv", sep='\t', encoding='utf-8',quoting=csv.QUOTE_NONE)
  ocr_df = ocr_df.dropna()
  ocr_df = ocr_df.drop(ocr_df[ocr_df.text.str.strip() == ''].index)
  text_output = ocr_df.text.tolist()
  doc_text = ' '.join(text_output)

  #read an image file for inference
  inference_image = Image.open(imag_path).convert('RGB')
  width, height = inference_image.size

  words = []
  for index,row in ocr_df.iterrows():
    word = {}
    origin_box = [row['left'],row['top'],row['left']+row['width'],row['top']+row['height']] 
    word['word_text'] = row['text']
    word['word_box'] = origin_box
    word['normalized_box'] = normalize_box(word['word_box'],width, height)
    words.append(word)

  boxlist = [word['normalized_box'] for word in words]
  wordlist = [word['word_text'] for word in words]

  encoding = inference_processor(inference_image,wordlist,boxes=boxlist,return_tensors="pt",padding="max_length", truncation=True) 
  for k,v in encoding.items():
    encoding[k] = v.to(device)

  model.eval()
  with torch.no_grad():
    inference_outputs = model(**encoding)
  inference_outputs.logits.shape

  raw_input_ids = encoding['input_ids'][0].tolist()
  predictions = inference_outputs.logits.argmax(-1).squeeze().tolist()
  token_boxes = encoding.bbox.squeeze().tolist()
  special_tokens = [inference_processor.tokenizer.cls_token_id, inference_processor.tokenizer.sep_token_id, inference_processor.tokenizer.pad_token_id]

  input_ids = [id for id in raw_input_ids if id not in special_tokens]
  predictions = [model.config.id2label[prediction] for i,prediction in enumerate(predictions) if not (raw_input_ids[i] in special_tokens)]
  actual_boxes = [box for i,box in enumerate(token_boxes) if not (raw_input_ids[i] in special_tokens )]

  assert(len(actual_boxes) == len(predictions))

  for word in words:
    word_labels = [] 
    token_labels = []
    word_tagging = None 
    for i,box in enumerate(actual_boxes,start=0):
      if compare_boxes(word['normalized_box'],box):
        if predictions[i] != 'O':
          word_labels.append(predictions[i][2:])
        else:
          word_labels.append('O')
        token_labels.append(predictions[i])
    if word_labels != []:
      word_tagging =  word_labels[0] if word_labels[0] != 'O' else word_labels[-1]
    else:
      word_tagging = 'O'
    word['word_labels'] = token_labels
    word['word_tagging'] = word_tagging

  filtered_words = [{'id':i,'text':word['word_text'],
                    'label':word['word_tagging'],
                    'box':word['word_box'],
                    'words':[{'box':word['word_box'],'text':word['word_text']}]} for i,word in enumerate(words) if word['word_tagging'] != 'O']

  merged_taggings = []
  for i,curr_word in enumerate(filtered_words):
    skip = False
    neighbors = lambda word:[neighbor for neighbor in filtered_words if mergable(word,neighbor)]
    for items in merged_taggings:
      for item in items:
        if item in neighbors(curr_word):
          skip = True
          break
      if skip:
        break
    if skip:
      continue
    merged_taggings.append(neighbors(curr_word))

  merged_words = []
  for i,merged_tagging in enumerate(merged_taggings):
    if len(merged_tagging) > 1:
      new_word = {}
      merging_word = " ".join([word['text'] for word in merged_tagging])
      merging_box = [merged_tagging[0]['box'][0]-5,merged_tagging[0]['box'][1]-10,merged_tagging[-1]['box'][2]+5,merged_tagging[-1]['box'][3]+10]
      new_word['text'] = merging_word
      new_word['box'] = merging_box
      new_word['label'] = merged_tagging[0]['label']
      new_word['id'] = filtered_words[-1]['id']+i+1
      new_word['words'] = [{'box':word['box'],'text':word['text']} for word in merged_tagging]
      merged_words.append(new_word)

  filtered_words.extend(merged_words)
  predictions = [word['label'] for word in filtered_words]
  actual_boxes = [word['box'] for word in filtered_words]
  unique_taggings = set(predictions)

  label2color = {f'{label}':f'rgb({random_color()[0]},{random_color()[1]},{random_color()[2]})' for label in unique_taggings}

  inference_image = Image.open(imag_path).convert('RGB')
  draw = ImageDraw.Draw(inference_image)
  font = ImageFont.load_default()
  taggings = {}
  for prediction, box in zip(predictions, actual_boxes):
      # predicted_label = iob_to_label(prediction).lower()
      draw.rectangle(box, outline=label2color[prediction])
      draw.text((box[0] + 10, box[1] - 10), text=prediction, fill=label2color[prediction], font=font)  

  doc_name = os.path.basename(imag_path)
  os.makedirs(output_path,exist_ok=True)
  inference_image.save(f"{output_path}/{img_name}_Output.png")
  dictionary = {"document name":doc_name,"document": doc_text , "form": filtered_words}
  with open(f"{output_path}/{img_name}_jsonOutput.json","w",encoding='utf8') as outfile:
      json.dump(dictionary, outfile,ensure_ascii=False)

In [52]:
test_dir = '/content/drive/MyDrive/intellectAI/inv_test/'
output_path = '/content/out'
model_path = '/content/drive/MyDrive/intellectAI/model/layoutlmv3_final.pth'
model= torch.load(model_path)
for img_name in os.listdir(test_dir):
  try:
    inference(img_name,test_dir,output_path,model)
  except Exception as e:
    print(img_name,e)